In [16]:
import pandas as pd
from langchain.llms import Ollama
from langchain.llms import BaseLLM
import json
import requests
import numpy as np

In [20]:
class Ollama(BaseLLM):
    base_url: str
    model: str
    prompt: str

    def _call(self, prompt, stop=None):
        try:
            response = requests.post(
                f"{self.base_url}/api/generate",  # Adjust the endpoint if necessary
                json={"model": self.model, "prompt": self.prompt}
            )
            response.raise_for_status()  # Raise an error for bad HTTP status codes
            raw_content = response.content.decode('utf-8')
            print("Raw Response content:", raw_content)  # Debugging
            
            # Handle multiple JSON objects
            response_json = None
            for part in raw_content.split('\n'):
                try:
                    response_json = json.loads(part)
                    break  # Stop after the first valid JSON object
                except json.JSONDecodeError:
                    continue
            
            if response_json:
                print("Response JSON:", response_json)  # Debugging
                return response_json.get('text', '')
            else:
                print("No valid JSON found in response.")
        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
        except ValueError as e:
            print(f"JSON decode error: {e}")
            print(f"Response content: {raw_content.to_numpy()}")
        return ""

    def _generate(self, prompts, stop=None):
        return [self._call(prompt, stop) for prompt in prompts]

    @property
    def _llm_type(self):
        return "ollama"

    @property
    def _identifying_params(self):
        return {"base_url": self.base_url, "model": self.model, "prompt": self.prompt}

In [21]:
prompt = """
Suppose you are an analyst who needs to take a user review of an application as input and extract all the app problems, and the corresponding user actions that the user takes to encounter the app problem. An app problem is defined as a single problem faced when interacting with an application. A user action is defined as a single interaction with the application. Show the outputs in the form of python list of dictionaries.

The list will contain dictionaries, each dictionary will have app problem as the key and a list of user actions in sequential order as values. It is based on the following scenarios: 

Scenario-1: A pair of user action and app problem exists. For that write the output as follows: 
[{'app problem-1': ['user action-1']}]

Scenario-2: An app problem may have multiple user actions in sequence that lead to that app problem. For that, write the output as follows: 
[{'app problem-1': ['user action-1', 'user action-2']}] 

Scenario 3: A user action can lead to multiple app problems. For that write the output as follows: 
[{'app problem-1': ['user action-1']}, {'app problem-2': ['user action-1']}] 

Scenario-4: There are no app problems with corresponding user action, or there are app problems with no user actions. For that, we leave it blank.

Just write according to the given format above and nothing more. Do not write any other generated tests when I give an input. Understood?
"""

In [22]:
ollama = Ollama(base_url='http://localhost:11434', model='llama2', prompt=prompt)

In [23]:
ollama("Facebook, i can't log in into my account it says “unable to log in expired session”, i already updated it clear datas anything, please fix it i need to log into my account, i can't log in Facebook please fix this bug")

TypeError: 'Ollama' object is not callable

In [25]:
import requests

class Ollama:
    def __init__(self, base_url, model, prompt):
        self.base_url = base_url
        self.model = model
        self.prompt = prompt

    def _prepare_payload(self, review, output):
        eos_token = "<|endoftext|>"  # Define the end of string token
        content = f"{self.prompt}\n### content:\n{review}\n\n### events refined:\n{output}{eos_token}"
        return {
            "model": self.model,
            "prompt": content
        }

    def get_response(self, review, output):
        payload = self._prepare_payload(review, output)
        response = requests.post(f"{self.base_url}/api/generate", json=payload)
        response.raise_for_status()  # Raise an error for bad status codes
        return response.json()

# Example usage:
ollama = Ollama(base_url='http://localhost:11434', model='llama2Scenerio', prompt=prompt)
review = "The app crashes every time I try to open the settings."
output = "[{'app problem-1': ['user action-1']}]"
response = ollama.get_response(review, output)
print(response)


JSONDecodeError: Extra data: line 2 column 1 (char 102)